In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

#每次可以输出多个变量
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
batch_size = 100
n_batch = mnist.train.num_examples // batch_size

In [5]:
image_x = len(mnist.train.images[0])
image_y = len(mnist.train.labels[0])

In [6]:
x = tf.placeholder(tf.float32, [None, image_x])
y = tf.placeholder(tf.float32, [None, image_y])

# dropout
keep_prob = tf.placeholder(tf.float32)

# 动态改变学习率
lr = tf.Variable(0.001, dtype=tf.float32)

In [7]:
# 原始
W = tf.Variable(tf.zeros([image_x, image_y]))
b = tf.Variable(tf.zeros([10]))
prediction = tf.nn.softmax(tf.matmul(x, W) + b)

In [8]:
# 优化
layer_cnt1 = 2000
layer_cnt2 = 2000
layer_cnt3 = 2000
layer_cnt4 = 2000
layer_cnt5 = 1000
bias = 0.1

# 初始化（权值使用截断的正态分布，偏置使用0.1）
weight_l1 = tf.Variable(tf.truncated_normal([image_x, layer_cnt1], stddev=0.1))
bias_l1 = tf.Variable(tf.zeros([layer_cnt1]) + bias)
input_l1 = tf.matmul(x, weight_l1) + bias_l1
output_l1 = tf.nn.tanh(input_l1)
l1_dropout = tf.nn.dropout(output_l1, keep_prob)

weight_l2 = tf.Variable(tf.truncated_normal([layer_cnt1, layer_cnt2], stddev=0.1))
bias_l2 = tf.Variable(tf.zeros([layer_cnt2]) + bias)
input_l2 = tf.matmul(l1_dropout, weight_l2) + bias_l2
output_l2 = tf.nn.tanh(input_l2)
l2_dropout = tf.nn.dropout(output_l2, keep_prob)

weight_l3 = tf.Variable(tf.truncated_normal([layer_cnt2, layer_cnt3], stddev=0.1))
bias_l3 = tf.Variable(tf.zeros([layer_cnt3]) + bias)
input_l3 = tf.matmul(l2_dropout, weight_l3) + bias_l3
output_l3 = tf.nn.tanh(input_l3)
l3_dropout = tf.nn.dropout(output_l3, keep_prob)

weight_l4 = tf.Variable(tf.truncated_normal([layer_cnt3, layer_cnt4], stddev=0.1))
bias_l4 = tf.Variable(tf.zeros([layer_cnt4]) + bias)
input_l4 = tf.matmul(l3_dropout, weight_l4) + bias_l4
output_l4 = tf.nn.tanh(input_l4)
l4_dropout = tf.nn.dropout(output_l4, keep_prob)

weight_l5 = tf.Variable(tf.truncated_normal([layer_cnt4, layer_cnt5], stddev=0.1))
bias_l5 = tf.Variable(tf.zeros([layer_cnt5]) + bias)
input_l5 = tf.matmul(l2_dropout, weight_l5) + bias_l5
output_l5 = tf.nn.tanh(input_l5)
l5_dropout = tf.nn.dropout(output_l5, keep_prob)

weight_l6 = tf.Variable(tf.truncated_normal([layer_cnt5, image_y], stddev=0.1))
bias_l6 = tf.Variable(tf.zeros([image_y]) + bias)
input_l6 = tf.matmul(l5_dropout, weight_l6) + bias_l6
output_l6 = tf.nn.softmax(input_l6)

prediction = output_l6

In [9]:
# loss = tf.reduce_mean(tf.square(y - prediction)) # 二次代价函数
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=prediction)) # 交叉熵代价函数
# optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1) # SGD
# optimizer = tf.train.AdadeltaOptimizer() # Adadelta
optimizer = tf.train.AdamOptimizer(lr) # Adam
train = optimizer.minimize(loss)

init = tf.global_variables_initializer()

In [10]:
# 准确率
correct_prediction = tf.equal(tf.argmax(y, axis=1), tf.argmax(prediction, axis=1))
acc = tf.reduce_mean(tf.cast(correct_prediction, dtype=tf.float32))

In [11]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

sess.run(init)
for epoch in range(70):
    _ = sess.run((tf.assign(lr, 0.001 * (0.95 ** epoch))))
    for batch in range(n_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        # 训练时dropout设为较小的浮点数，最终运行模型时依然设为1
        sess.run(train, feed_dict={x:batch_xs, y:batch_ys, keep_prob:0.5})
#     print('Iter ' + str(epoch) + '->train acc:' + str(sess.run(acc, feed_dict={x:mnist.train.images, y:mnist.train.labels, keep_prob:0.3})) + ',test acc:' + str(sess.run(acc, feed_dict={x:mnist.test.images, y:mnist.test.labels, keep_prob:0.3})))
    print('Iter ' + str(epoch) + '->train acc:' + str(sess.run(acc, feed_dict={x:mnist.train.images, y:mnist.train.labels, keep_prob:1.0})) + ',test acc:' + str(sess.run(acc, feed_dict={x:mnist.test.images, y:mnist.test.labels, keep_prob:1.0})))
#     print(' ')
sess.close()

Iter 0->train acc:0.906727,test acc:0.9105
Iter 1->train acc:0.913109,test acc:0.9152
Iter 2->train acc:0.925909,test acc:0.9285
Iter 3->train acc:0.930909,test acc:0.9335
Iter 4->train acc:0.935055,test acc:0.9383
Iter 5->train acc:0.942018,test acc:0.9429
Iter 6->train acc:0.942964,test acc:0.9414
Iter 7->train acc:0.947618,test acc:0.947
Iter 8->train acc:0.9494,test acc:0.9472
Iter 9->train acc:0.950727,test acc:0.9489
Iter 10->train acc:0.953418,test acc:0.9503
Iter 11->train acc:0.954291,test acc:0.9516
Iter 12->train acc:0.956709,test acc:0.9525
Iter 13->train acc:0.957364,test acc:0.9536
Iter 14->train acc:0.9582,test acc:0.9552
Iter 15->train acc:0.961309,test acc:0.9583
Iter 16->train acc:0.961455,test acc:0.957
Iter 17->train acc:0.963036,test acc:0.9591
Iter 18->train acc:0.9636,test acc:0.9591
Iter 19->train acc:0.966182,test acc:0.9619
Iter 20->train acc:0.966782,test acc:0.963
Iter 21->train acc:0.967491,test acc:0.9631
Iter 22->train acc:0.967964,test acc:0.964
Iter 23-

## 调优结论
- 交叉熵代价函数比二次代价函数使模型更快收敛，且准确率更高
- 增加dropout会使迭代速度变慢，但是和train的准确率差距变小

## 防止过拟合
- 增加数据集
- 正则化
- Dropout

## Optimizer
![SGD](./img/sgd.png)
- SGD:朝着梯度的反方向（优化速度最快）优化w和b。


![Momentum](./img/momentum.png)
- Momentum:在SGD基础上增加一个动力项（通常设成0.9），当前权值改变受到上一次权值改变影响，加快收敛速度。


![NAG](./img/nag.png)
- NAG(Nesterov accelerated gradient):在计算参数时也减去动力项，使收敛更加精确。


![Adagrad](./img/adagrad.png)
- Adagrad:比较常见的数据用较小的学习率调整参数，罕见的数据使用较大的学习率调整参数，适用于参数不平衡的数据集，但是随着迭代次数增多会使学习率逐渐趋向于0。


![RMS](./img/RMSprop.png)
- RMSprop(Root Mean Square均方根):前t次梯度平方的均值加上当前梯度的平方根作为学习率的分母，不会出现学习率减小的问题。


![Adadelta](./img/adadelta.png)
- Adadelta:不用设置学习率也能得到一个很好的效果。


![Adam](./img/adam.png)
- Adam:存储之前衰减的平方梯度，自动调整学习率。

![Compare](./img/compare.png)